In [4]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import DepthwiseConv2D as _BaseDepthwise
from PIL import Image, ImageOps
import time

# 1) custom DepthwiseConv2D 정의 (config에서 groups 키워드를 pop)
class DepthwiseConv2D(_BaseDepthwise):
    @classmethod
    def from_config(cls, config):
        config.pop('groups', None)
        return super().from_config(config)

# 2) custom_objects 에 등록해서 모델 로드
model = load_model(
    'keras_model.h5',
    compile=False,
    custom_objects={'DepthwiseConv2D': DepthwiseConv2D}
)

# 3) labels.txt 로드
with open('labels.txt', 'r') as f:
    labels = [line.strip() for line in f.readlines()]

/Users/jongphilkim/Documents/GitHub/TeachableMachine_Project/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
from google.colab import files
import os

# 모델 파일 업로드
print("keras_model.h5 파일을 업로드해주세요.")
uploaded = files.upload()
model_path = next(iter(uploaded))

# labels.txt 파일 업로드  
print("\nlabels.txt 파일을 업로드해주세요.")
uploaded = files.upload()
labels_path = next(iter(uploaded))

# 모델 로드
model = load_model(
    model_path,
    compile=False, 
    custom_objects={'DepthwiseConv2D': DepthwiseConv2D}
)

# labels.txt 로드 및 출력
with open(labels_path, 'r') as f:
    labels = [line.strip() for line in f.readlines()]
    
print("\n라벨 목록:")
for i, label in enumerate(labels):
    print(f"{i}: {label}")

ModuleNotFoundError: No module named 'google.colab'

: 

In [5]:
# --- 이하 기존 코드 이어서 사용 ---
def preprocess_frame(frame):
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
    arr = np.asarray(image).astype(np.float32)
    normalized = (arr / 127.5) - 1
    return np.expand_dims(normalized, axis=0)

def predict_mask_status(frame):
    proc = preprocess_frame(frame)
    preds = model.predict(proc, verbose=0)
    idx = np.argmax(preds)
    return labels[idx], float(preds[0][idx])


In [6]:
cap = cv2.VideoCapture(0)
no_mask_count = 0
warning_threshold = 3

while True:
    ret, frame = cap.read()
    if not ret:
        break

    label, conf = predict_mask_status(frame)
    color = (0,255,0) if label=='mask' else (0,0,255)

    cv2.putText(frame, f"{label}:{conf:.2f}", (10,30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    if label=='no_mask':
        no_mask_count += 1
        msg = (f"경고: 마스크 착용! ({no_mask_count}/{warning_threshold})"
               if no_mask_count < warning_threshold
               else "상습 위반자입니다. 고발 조치 예정!")
        cv2.putText(frame, msg, (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
    else:
        no_mask_count = 0

    cv2.imshow('Mask Detection', frame)
    if cv2.waitKey(1)&0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()